In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_kr = pd.read_parquet("../../../data/dataframes/정신력남자.parquet")
df_euw = pd.read_parquet("../../../data/dataframes/Don_Noway.parquet")
df_noway2u = pd.read_parquet("../../../data/dataframes/noway2u.parquet")
df_broses = pd.read_parquet("../../../data/dataframes/TRM_BROSES.parquet")
df_nicigeddon = pd.read_parquet("../../../data/dataframes/TRM_Nicigeddon.parquet")

In [ ]:
df_kr.shape
print(df_euw.shape[0] + df_noway2u.shape[0])

df_euw = pd.concat([df_euw, df_noway2u]).sort_values(by="gameCreation", ascending=False)
df_all = pd.concat([df_euw, df_noway2u, df_kr]).sort_values(by="gameCreation", ascending=False)
df_all.shape

In [ ]:
df_all.tail()

#### Champion Game time

In [ ]:
filt = (df_kr["championName"].value_counts() >= 5)
filtered_df = df_kr[df_kr["championName"].isin(filt[filt].index)]

filtered_df.groupby("championName")["gameDuration"].mean() / 60

#### Average Game time

In [ ]:
avg_game_time_korea = df_kr["gameDuration"].mean() / 60
avg_game_time_euw = df_euw["gameDuration"].mean() / 60

print(avg_game_time_euw, avg_game_time_korea)

In [ ]:
# Data for plotting
regions = ['Korea', 'EUW']
avg_game_times = [avg_game_time_korea, avg_game_time_euw]

# Plotting as a bar chart
plt.bar(regions, avg_game_times, color=['blue', 'green'])
plt.xlabel('Region')
plt.ylabel('Average Game Time (minutes)')
plt.title('Average Game Time for Korea and EUW')
plt.show()


#### Game end by minute

In [ ]:
# Binning und Häufigkeitszählung
bins = range(0, df_kr['gameDuration'].max() // 60 + 1, 2)
df_kr['duration_bin'] = pd.cut(df_kr['gameDuration'] / 60, bins=bins, right=False)
bin_counts = df_kr['duration_bin'].value_counts().sort_index()

# Plotting als Liniendiagramm
bin_counts.plot(kind='line', grid=True)
plt.xlabel('Game Duration (minutes)')
plt.ylabel('Frequency')
plt.title('Distribution of Game Duration Bins')

# x-Achsenbeschriftung anpassen
plt.xticks(ticks=range(len(bin_counts.index)))

bins = range(0, df_euw['gameDuration'].max() // 60 + 1, 2)
df_euw['duration_bin'] = pd.cut(df_euw['gameDuration'] / 60, bins=bins, right=False)
bin_counts = df_euw['duration_bin'].value_counts().sort_index()

# Plotting als Liniendiagramm
bin_counts.plot(kind='line', grid=True)
plt.xlabel('Game Duration (minutes)')
plt.ylabel('Frequency')
plt.title('Distribution of Game Duration Bins')

# x-Achsenbeschriftung anpassen
plt.xticks(ticks=range(len(bin_counts.index)))

plt.show()

bin_counts

#### Average game time per patch

In [ ]:
df_euw.sort_values(by="gameCreation", inplace=True)

df_euw.head(1)

In [ ]:
# Convert 'gameVersion' to a Categorical data type for custom sorting
df_euw['gameVersion'] = pd.Categorical(df_euw['gameVersion'], categories=df_euw['gameVersion'].unique(), ordered=True)

# Sort the DataFrame by 'gameVersion' in ascending order
df_euw = df_euw.sort_values(by='gameVersion', ascending=True)

# Group by "gameVersion" and calculate the mean
mean_game_duration = df_euw.groupby("gameVersion")["gameDuration"].mean() / 60

ax = mean_game_duration.plot(kind="line")
plt.xlabel("Game Version")
plt.ylabel("Mean Game Duration (minutes)")
plt.title("Mean Game Duration by Game Version")
plt.grid()

# Add x-axis labels for every second categorical value
ax.set_xticks(range(0, len(mean_game_duration), 2))
ax.set_yticks(range(20, 31, 1))
ax.set_xticklabels(mean_game_duration.index[::2], rotation=45, ha='right')

plt.show()

#### Win percentage with increasing gold lead

In [ ]:
team_gold_diff_df = df_euw.filter(like="totalTeamGoldDiff", axis="columns").copy()
team_gold_diff_df.loc[:, "win"] = df_euw["win"]

team_gold_diff_df["totalTeamGoldDiff@30"].notna().value_counts()

In [ ]:
team_gold_adv_dict = {}

START = 0
STOP = 15001
STEP = 250

bins = range(START, STOP, STEP)

team_gold_adv_dict = {
    bin: {
        "games": 0,
        "wins": 0
    } for bin in range(START, STOP, STEP)
}

for bin in bins:
    for index, gold_diffs in team_gold_diff_df.iterrows():
        for gold_diff in gold_diffs:
            if bin <= gold_diff <= bin + STEP:
                team_gold_adv_dict[bin]["games"] += 1
                if team_gold_diff_df.loc[index]["win"]:
                    team_gold_adv_dict[bin]["wins"] += 1
                break

In [ ]:
team_gold_dis_dict = {}

START = 0
STOP = -15001
STEP = -250

bins = range(START, STOP, STEP)

team_gold_dis_dict = {
    bin: {
        "games": 0,
        "wins": 0
    } for bin in range(START, STOP, STEP)
}

for bin in bins:
    for index, gold_diffs in team_gold_diff_df.iterrows():
        for gold_diff in gold_diffs:
            if bin + STEP <= gold_diff <= bin:
                team_gold_dis_dict[bin]["games"] += 1
                if team_gold_diff_df.loc[index]["win"]:
                    team_gold_dis_dict[bin]["wins"] += 1
                break

In [ ]:
team_gold_dis_win_rate_df = pd.DataFrame(team_gold_dis_dict).T
team_gold_dis_win_rate_df["winrate"] = team_gold_dis_win_rate_df["wins"] / team_gold_dis_win_rate_df["games"] * 100
team_gold_dis_win_rate_df.set_index(team_gold_dis_win_rate_df.index * -1)

In [ ]:
team_gold_adv_win_rate_df = pd.DataFrame(team_gold_adv_dict).T
team_gold_adv_win_rate_df["winrate"] = team_gold_adv_win_rate_df["wins"] / team_gold_adv_win_rate_df["games"] * 100

In [ ]:
START = 0
STOP = 15001
STEP = 250

plt.plot(range(START, STOP, STEP), team_gold_adv_win_rate_df["winrate"])
plt.plot(range(START, STOP, STEP), team_gold_dis_win_rate_df["winrate"])
plt.axhline(y=50, color='black', linestyle='-', label='50 % winrate')
plt.grid()
plt.xticks(range(START, STOP, STEP * 4), rotation=25)
plt.yticks(range(0, 101, 10))
plt.ylabel("Winrate in %")
plt.xlabel("Team Gold Difference")
plt.legend(["Winrate with gold advantage", "Winrate with gold disadvantage", "50 % winrate reference"])
plt.title("Winrate based on Team Gold Difference")

plt.show()

#### Players in game

In [ ]:
df_broses.sort_values(by="gameCreation").head()

In [ ]:
import requests

# participants_df = df_euw.filter(like="participant")
# participants_df = df_kr.filter(like="participant")
participants_df = df_broses.filter(like="participant")
for puuid, amount in participants_df.unstack().value_counts().head(10).items():
    print(amount, requests.get(f"https://europe.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key=RGAPI-284e5f95-5b7d-4e15-a9dc-705170bb98e8").json()["gameName"])

#### All Games combined

In [ ]:
result = (
    df_all.query('laneOpponent == "Heimerdinger"')
    .groupby('championName')
    .agg(games=('win', 'count'), winrate=('win', 'mean'))
    .reset_index()
)

result


plt.scatter(result["games"], result["winrate"])
plt.axhline(y=0.5)
plt.axvline(x=result["games"].max() / 2)

plt.xticks(range(0, result["games"].max() + 1, 1))

plt.show()

In [ ]:
half_games = result["games"].max() // 2 + 1

best_champion = (
    result.query('games >= @half_games')
    .sort_values(by='winrate', ascending=False)
    .iloc[0]['championName']
)

print("Best Champion:", best_champion)

#### Noways Mathe mann champion based on his record for last 10 Lane Opponents

In [ ]:
mathe_mann = {"laneOpponent": [], "championName": []}

for laneOpponent in df_all["laneOpponent"].unique():
    
    mathe_mann['laneOpponent'].append(laneOpponent)

    print(laneOpponent, end=" -> ")

    result = (
    df_all.query('laneOpponent == @laneOpponent')
    .groupby('championName')
    .agg(games=('win', 'count'), winrate=('win', 'mean'))
    .reset_index()
    )

    half_games = result["games"].max() // 2 + 1

    best_champion = (
    result.query('games >= @half_games')
    .sort_values(by='winrate', ascending=False)
    .iloc[0]['championName']
    )

    mathe_mann['championName'].append(best_champion)

    print(best_champion)

mathe_matchups = pd.DataFrame(mathe_mann)

In [ ]:
mathe_matchups.shape

#### Mathe mann for all matchups played across the team

In [ ]:
result = df_all.groupby(["allyTopPick", "enemyTopPick"]).agg(
    games=('win', 'size'),
    winrate_percentage=('win', lambda x: x.mean() * 100)
)

result.sort_values(by="games", ascending=False).head()

In [ ]:
plt.scatter(result["games"], result["winrate_percentage"])

In [ ]:
def shark(start_champion):
    champions = [start_champion]
    current_champion = start_champion
    while True:
        counter = get_best_matchup(current_champion)
        if counter in champions:
            break
        champions.append(counter)
        current_champion = counter
    return champions

def get_best_matchup(champion):
    half_games = result.loc[champion, :]["games"].max() // 2 + 1
    return result.query("(index.get_level_values('allyTopPick') == @champion) & games >= @half_games").sort_values(by="winrate_percentage", ascending=False).index[0][1]

In [ ]:
champions_list = shark("Ahri")

for index, champion in enumerate(champions_list):
    print(champion, end="")
    if index < len(champions_list) - 1:
        print(" < ", end="")

#### Matchups based on winrate

In [ ]:
df_all.query("championName == 'Ahri'")[["laneOpponent", "win"]].groupby("laneOpponent").agg(
        games=('win', 'size'),
        winrate=('win', 'mean')
    ).sort_values(by="games", ascending=False).reset_index().head()


#### Matchups based on individual gold diff at minute 10

In [ ]:
df_all["gold_adv_@10"] = df_all["gold_diff@10"] > 0

In [ ]:
df_all.query("championName == 'Ahri'")[["laneOpponent", "gold_adv_@10"]].groupby("laneOpponent").agg(
        games=('gold_adv_@10', 'size'),
        perc_gold_lead_at_10=('gold_adv_@10', 'mean')
    ).sort_values(by="games", ascending=False).reset_index().head()

In [267]:
import pandas as pd

champion = "Viktor"

# Assuming you have already calculated games and winrate
games_and_winrate = df_all.query("championName == @champion")[["laneOpponent", "win"]].groupby("laneOpponent").agg(
    games=('win', 'size'),
    winrate=('win', 'mean')
).reset_index()

# Assuming you have already calculated games and perc_gold_lead_at_10
games_and_gold_adv_at_10 = df_all.query("championName == @champion")[["laneOpponent", "gold_adv_@10"]].groupby("laneOpponent").agg(
    games=('gold_adv_@10', 'size'),
    perc_gold_lead_at_10=('gold_adv_@10', 'mean')
).reset_index()

# Merge the two DataFrames on "laneOpponent"
combined_result = pd.merge(games_and_winrate, games_and_gold_adv_at_10, on=["laneOpponent", "games"]).sort_values(by="games", ascending=False)

combined_result.head()

,laneOpponent,games,winrate,perc_gold_lead_at_10
44,Sylas,18,0.555556,0.611111
58,Yone,15,0.600000,0.666667
23,KSante,7,0.714286,0.714286
36,Renekton,7,0.714286,0.571429
20,Irelia,6,0.333333,0.666667


In [272]:
import numpy as np

# Assuming you have a DataFrame named combined_result with columns winrate and perc_gold_lead_at_10

conditions = [
    (combined_result['perc_gold_lead_at_10'] >= 0.50) & (combined_result['winrate'] >= 0.50),
    (combined_result['perc_gold_lead_at_10'] >= 0.50) & (combined_result['winrate'] < 0.50),
    (combined_result['perc_gold_lead_at_10'] < 0.50) & (combined_result['winrate'] >= 0.50),
    (combined_result['perc_gold_lead_at_10'] < 0.50) & (combined_result['winrate'] < 0.50),
]

# choices = ['win lane win game', 'win lane lose game', 'lose lane win game', 'lose lane lose game']
choices = [2, 1, -1, -2]

combined_result['match_up_category'] = np.select(conditions, choices, default='other')

combined_result.query("match_up_category == '-2'")

,laneOpponent,games,winrate,perc_gold_lead_at_10,match_up_category
28,Malphite,3,0.333333,0.333333,-2
11,Ekko,3,0.000000,0.333333,-2
9,Darius,3,0.000000,0.000000,-2
62,Zoe,3,0.333333,0.000000,-2
55,Xerath,1,0.000000,0.000000,-2
29,Neeko,1,0.000000,0.000000,-2
48,TwistedFate,1,0.000000,0.000000,-2
38,Rumble,1,0.000000,0.000000,-2
